In [2]:
# libraries
#!pip install rpy2
#!pip install pandas
#!pip install keras

    100% |████████████████████████████████| 184kB 518kB/s 
  Running setup.py bdist_wheel for rpy2 ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done
  Stored in directory: /Users/Kozodoi/Library/Caches/pip/wheels/23/9e/ee/0e5f6a00aafef9935d40ebf7657278220139f0101321e30d07
Successfully built rpy2


In [5]:
# libraries
#import rpy2.robjects as robjects
import pandas as pd
import keras
from keras.layers import Input, Dense, Embedding, concatenate, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.regularizers import l2
from keras.optimizers import Adam

Using Theano backend.


In [6]:
# load the data
#path = "/Users/Kozodoi/Documents/Competitions/DSG_2017/"
path = "/Users/maj/Dropbox/DSG17/DSG_2017/"
data   = pd.read_csv(path + "data/data_full.csv")

In [7]:
known = data.query("dataset == 'train'")
unknown = data.query("dataset == 'unknown'")

In [8]:
# keep only Flow songs in the data
# this proves to predict better, but some information is lost
known = known.query("listen_type == 1")
known.shape

(1640286, 40)

In [9]:
# Prepare the data
# Create a placeholder for the IDs new in the test data
newUsers   = list(unknown.user_id[~unknown.user_id.isin(known.user_id)])       + list(known.user_id.value_counts().keys()[known.user_id.value_counts()     == 1])
newSongs   = list(unknown.media_id[~unknown.media_id.isin(known.media_id)])    + list(known.media_id.value_counts().keys()[known.media_id.value_counts()   == 1])[-1000:]
newArtists = list(unknown.artist_id[~unknown.artist_id.isin(known.artist_id)]) + list(known.artist_id.value_counts().keys()[known.artist_id.value_counts() == 1])[-1000:]
newGenres  = list(unknown.genre_id[~unknown.genre_id.isin(known.genre_id)])    + list(known.genre_id.value_counts().keys()[known.genre_id.value_counts()   == 1])[-1000:]

# In particular, assign IDs from 0 to N for users and songs
# Use enumerate() to create a list of the new and original IDs
users   = list(enumerate([i for i in known.user_id.unique()   if i not in newUsers]))
songs   = list(enumerate([i for i in known.media_id.unique()  if i not in newSongs]))
artists = list(enumerate([i for i in known.artist_id.unique() if i not in newArtists]))
genres  = list(enumerate([i for i in known.genre_id.unique()  if i not in newGenres]))

# Create a dictionary with old IDs to new IDs
userid2idx   = {o:i for i,o in users}
songid2idx   = {o:i for i,o in songs}
artistid2idx = {o:i for i,o in artists}
genreid2idx  = {o:i for i,o in genres}

# Update with new/rare entries
userid2idx.update({o:(max(userid2idx.values())+1)     for o in newUsers})
songid2idx.update({o:(max(songid2idx.values())+1)     for o in newSongs})
artistid2idx.update({o:(max(artistid2idx.values())+1) for o in newArtists})
genreid2idx.update({o:(max(genreid2idx.values())+1)   for o in newGenres})

# Create id variable with the new IDs
known['userIdx']   = known.user_id.apply(lambda x:       userid2idx[x])
known['songIdx']   = known.media_id.apply(lambda x:      songid2idx[x])
known['artistIdx'] = known.artist_id.apply(lambda x:     artistid2idx[x])
known['genreIdx']  = known.genre_id.apply(lambda x:      genreid2idx[x])
unknown['userIdx']   = unknown.user_id.apply(lambda x:   userid2idx[x])
unknown['songIdx']   = unknown.media_id.apply(lambda x:  songid2idx[x])
unknown['artistIdx'] = unknown.artist_id.apply(lambda x: artistid2idx[x])
unknown['genreIdx']  = unknown.genre_id.apply(lambda x:  genreid2idx[x])

/Users/maj/anaconda/envs/deeplearning/lib/python3.6/site-packages/ipykernel/__main__.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/maj/anaconda/envs/deeplearning/lib/python3.6/site-packages/ipykernel/__main__.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/maj/anaconda/envs/deeplearning/lib/python3.6/site-packages/ipykernel/__main__.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [10]:
# partition train/test data: last 3 somgs per user go to validation
# model predicts better if trained on full known sample without validation
# but last 3 song per user give a fair estimate of accuracy on unknown data
ts = known.groupby(["userIdx"]).tail(3)
tr = known.groupby(["userIdx"], group_keys=False).apply(lambda x: x[:-3])

# move songs that appear only in ts to tr
strayObs = ts.songIdx.isin(tr.songIdx) & ts.userIdx.isin(tr.userIdx) & ts.artistIdx.isin(tr.artistIdx) & ts.genreIdx.isin(tr.genreIdx)
tr = tr.append(ts[~strayObs])
ts = ts[strayObs]

In [14]:
print(tr.shape)
print(ts.shape)

(1628712, 44)
(11574, 44)


In [ ]:
# create an input layer with one row of IDs
user_in   = Input(shape = (1,), dtype='int64', name = "user_in")
song_in   = Input(shape = (1,), dtype='int64', name = "song_in")
artist_in = Input(shape = (1,), dtype='int64', name = "artist_in")
genre_in  = Input(shape = (1,), dtype='int64', name = "genre_in")
data_in = Input(shape = (44,), name = "data_in") # Must select right columns! 44 is too high..

# Create an embedding assigning k latent factors to each ID
# These will be optimized
# A regulariztaion is added to avoid very large weights
n_users   = tr.userIdx.nunique()
n_songs   = tr.songIdx.nunique()
n_artists = tr.artistIdx.nunique()
n_genres  = tr.genreIdx.nunique()
u = Embedding(n_users,   100, input_length=1, embeddings_regularizer=l2(1e-5))(user_in)
s = Embedding(n_songs,   100, input_length=1, embeddings_regularizer=l2(1e-5))(song_in)
a = Embedding(n_artists, 50, input_length=1, embeddings_regularizer=l2(1e-5))(artist_in)
g = Embedding(n_genres,  50, input_length=1, embeddings_regularizer=l2(1e-5))(genre_in)

# Specify what to do with the layers
# We want to multiply them into a 'rating' matrix
x = concatenate([u, s])
#x = concatenate([u, s, data_in])
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
#x = Dropout(0.5)(Dense(128, activation='relu')(x))
#x = BatchNormalization()(x)
x = Dense(64, activation='relu')(x) 
x = Dropout(0.5)(x)
x = Dense(1, activation = "sigmoid")(x)
#x = merge([x, ub], mode = 'sum')
#x = merge([x, sb], mode = 'sum') # Can this be included in the line above?

# Then we specify the model that we want to use
model = Model([user_in, song_in], x) # 
model.compile(Adam(0.001), loss="binary_crossentropy", metrics = ['accuracy'])

In [ ]:
# run the estimations
model.fit([tr.userIdx, tr.songIdx], tr.is_listened,  #
validation_data = ([ts.userIdx, ts.songIdx], ts.is_listened),
batch_size = 22814, epochs = 10)

In [13]:
# predict on unlabelled set
pred = pd.DataFrame()
pred["sample_id"] = unknown.sample_id
pred["is_listened"] = model.predict([unknown.userIdx, unknown.songIdx, unknown.artistIdx, unknown.genreIdx])
pred.head(5)

,sample_id,is_listened
0,0,0.981744
1,1,0.783598
2,2,0.759315
3,3,0.388151
4,4,0.930634


In [14]:
# adding naive submission
naive = pd.read_csv(path + "submissions/naive_ratio_user.csv")
pred_mean = pred
pred_mean["is_listened"] = (pred["is_listened"] + naive["is_listened"])/2
pred_mean.head(5)

,sample_id,is_listened
0,0,0.990872
1,1,0.778805
2,2,0.819416
3,3,0.496504
4,4,0.911219


In [15]:
# saving submissions
pred.to_csv(path + "submissions/deep_128_64_flow.csv", index = False)
pred.to_csv(path + "submissions/deep_128_64_flow_plus_ratio_user.csv", index = False)